In [199]:

import pandas as pd
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForMultipleChoice, TrainingArguments, Trainer, BitsAndBytesConfig
from accelerate import Accelerator
import peft
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import datetime
import numpy as np


In [200]:
train = pd.read_csv('train.csv')

In [201]:
examples = pd.read_csv('60k_data/all_12_with_context2.csv')

In [202]:

sentences = [example for example in examples['prompt']]

In [203]:
sentences.extend(train['prompt'].tolist())

In [204]:
len(sentences) - 200

60347

In [206]:
from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
# print(embeddings)

In [207]:
from sentence_transformers import util

In [208]:
# embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
# cosine_scores = util.cos_sim(embeddings, embeddings)
paraphrases = util.paraphrase_mining(model, sentences,top_k=20)


# embed_src = model.encode(train['prompt'][0], convert_to_tensor=True)

In [211]:
similar_sentences = []

for paraphrase in paraphrases:
    score, i, j = paraphrase
    if (i in range(60347,len(sentences))) and (j < 60347):
        if .25<= score <= .90:
            # print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
            similar_sentences.append(sentences[j])


In [214]:
len(similar_sentences)

281

In [212]:
examples_2 = pd.read_csv('60k_data/train_with_context2.csv')

In [191]:
final_similar_questions = pd.concat([examples,examples_2])

In [187]:
len(list(set(similar_sentences)))

424

In [194]:
final_similar_questions=final_similar_questions.loc[final_similar_questions['prompt'].isin(list(set(similar_sentences)))]

In [198]:
final_similar_questions=final_similar_questions.loc[final_similar_questions.duplicated()==False]